### Run Model

#### CPU Run

In [2]:
import onnxruntime
import numpy as np
import torch
from timeit import default_timer as timer

# Specify the path to the quantized ONNZ Model
# onnx_model_path = "models/helloworld_quantized.onnx"
onnx_model_path = "./models/detector.onnx"

#Dummy input
in_shape = [1, 3, 608, 800]
input_data = torch.rand(in_shape)


cpu_options = onnxruntime.SessionOptions()

# Create Inference Session to run the quantized model on the CPU
cpu_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['CPUExecutionProvider'],
    sess_options=cpu_options,
)

def to_numpy(tensor):
            if tensor.requires_grad:
                return tensor.detach().cpu().numpy()
            return tensor.cpu().numpy()

ort_inputs = {cpu_session.get_inputs()[0].name: to_numpy(input_data)}

start = timer()
cpu_results = cpu_session.run(None, ort_inputs)
cpu_total = timer() - start

print(cpu_total)


0.4483639000000039


#### IPU Run

Now, we'll run it on the IPU and time the execution so that we can compare the results with the CPU.

In [3]:
# Compile and run

# Point to the config file path used for the VitisAI Execution Provider
config_file_path = "./vaip_config.json"
onnx_model_path = "models/detector.onnx"

aie_options = onnxruntime.SessionOptions()

aie_session = onnxruntime.InferenceSession(
    onnx_model_path,
    providers = ['VitisAIExecutionProvider'],
    sess_options=aie_options,
    provider_options=[{'config_file': config_file_path}]
)

start = timer()
ryzen_outputs = aie_session.run(None, ort_inputs)
aie_total = timer() - start

Let's gather our results and see what we have

In [4]:
print(f"Ryzen Results: {ryzen_outputs}")
print(f"CPU Results: {cpu_results}")

print(f"CPU Total Time: {cpu_total}")
print(f"IPU Total Time: {aie_total}")

Ryzen Results: [array([[[[0.00102132, 0.00022868],
         [0.0163164 , 0.00217931],
         [0.00102132, 0.00022868],
         ...,
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868]],

        [[0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         ...,
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868]],

        [[0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         ...,
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868]],

        ...,

        [[0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         ...,
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868],
         [0.00102132, 0.00022868]],

        [[0.00259171, 0.00160456],
         [0.00102132, 0.0002286